<a href="https://colab.research.google.com/github/Arctic-64/CE888/blob/main/Lab_3/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sklearn

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor, BayesianRidge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab3/bank-additional-full.csv", delimiter = ";")

print(df.shape)

df.drop('duration', inplace=True, axis=1)

df = df.dropna()

print(df.shape)

df.head()

(41188, 21)
(41188, 20)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [ ]:
df = df.apply(LabelEncoder().fit_transform)

features = ["age", "job", "marital", "education", "default", "housing", "loan", "contact", "month", "day_of_week", "campaign", "pdays", "previous", "poutcome", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"]

x_df = df[features].copy()

x_df = pd.get_dummies(x_df)

y_df = df["y"].copy()

print(x_df.head())

print(y_df.head())


x = x_df.values
y = y_df.values

print(x)
print(y)
print(df)

clf = DecisionTreeRegressor()
clf.fit(x, y)
print("Mean Squared Error ", mse(y, clf.predict(x)))

   age  job  marital  ...  cons.conf.idx  euribor3m  nr.employed
0   39    3        1  ...             16        287            8
1   40    7        1  ...             16        287            8
2   20    7        1  ...             16        287            8
3   23    0        1  ...             16        287            8
4   39    7        1  ...             16        287            8

[5 rows x 19 columns]
0    0
1    0
2    0
3    0
4    0
Name: y, dtype: int64
[[ 39   3   1 ...  16 287   8]
 [ 40   7   1 ...  16 287   8]
 [ 20   7   1 ...  16 287   8]
 ...
 [ 39   5   1 ...   0 171   0]
 [ 27   9   1 ...   0 171   0]
 [ 57   5   1 ...   0 171   0]]
[0 0 0 ... 0 1 0]
       age  job  marital  education  ...  cons.conf.idx  euribor3m  nr.employed  y
0       39    3        1          0  ...             16        287            8  0
1       40    7        1          3  ...             16        287            8  0
2       20    7        1          3  ...             16        287     

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, train_size=None, random_state=50, shuffle=True, stratify=y)
print("x_train", x_train)
print("y_train", y_train)
print("x_test", x_test)
print("y_test", y_test)

x_train [[  5   8   2 ...  25  60   3]
 [ 19   0   1 ...   6 305  10]
 [ 20   0   1 ...   3 216   6]
 ...
 [ 39   4   1 ...  16 287   8]
 [ 30   7   1 ...  17 310  10]
 [ 16   0   1 ...  16 291   8]]
y_train [1 0 1 ... 0 0 0]
x_test [[ 24   1   0 ...   8 306  10]
 [ 28   0   1 ...   3 221   6]
 [ 21   9   0 ...  17 307  10]
 ...
 [ 28   1   1 ...   6 306  10]
 [ 20   1   1 ...  16 289   8]
 [ 31   1   1 ...  17 307  10]]
y_test [0 0 0 ... 0 0 0]


In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=3)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
scores = cross_val_score(rf, x_train, y_train, cv=10, scoring=make_scorer(f1_score))
print(scores.mean(),scores.std())
print(f1_score(y_test, y_pred))
feature_imp = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
print(feature_imp)
param_grid = {
                 'n_estimators': [5, 10, 15, 20],
                 'max_depth': [2, 5, 7, 9]
             }
grid_rf = GridSearchCV(rf, param_grid, cv=10)
grid_rf.fit(x_train, y_train)
print(grid_rf.best_estimator_)

0.27556083254714575 0.04160962624817423
0.29982964224872233
euribor3m         0.255460
nr.employed       0.237438
pdays             0.139777
poutcome          0.117352
cons.conf.idx     0.072363
emp.var.rate      0.056787
cons.price.idx    0.047119
month             0.025095
age               0.018692
previous          0.015950
contact           0.007930
default           0.002829
day_of_week       0.001299
campaign          0.001104
job               0.000345
education         0.000199
marital           0.000103
loan              0.000101
housing           0.000057
dtype: float64
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimato